In [4]:
"""
XGBoost Balanced Pipeline - Optimal Performance in ~30-45 minutes
Two-stage tuning with reduced Stage 1 grid
Output: C:/Users/wdkal/Downloads/IE421_XGBOOST_DATA/

Run: python xgboost_balanced.py
"""

import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix
import joblib
import json
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Define paths
RAW_DATA_PATH = Path("C:/Users/wdkal/iex_data/book_snapshots")
OUTPUT_DIR = Path("C:/Users/wdkal/Downloads/IE421_XGBOOST_DATA")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

#####################################
# 1. Load order book data
#####################################
def fetch_orderbook_data(dates=['20251020', '20251021', '20251022', '20251023', '20251024']):
    """Load IEX order book snapshots from multiple dates."""
    print("="*60)
    print("STEP 1: LOADING RAW ORDER BOOK DATA")
    print("="*60)
    dfs = []
    for date in dates:
        file_path = RAW_DATA_PATH / f'{date}_book_updates.csv.gz'
        print(f"  Loading {date}...")
        df = pd.read_csv(file_path, compression='gzip')
        df['date'] = date
        df['COLLECTION_TIME'] = pd.to_datetime(df['COLLECTION_TIME'])
        df = df.set_index('COLLECTION_TIME')
        df = df.between_time("14:30", "21:00")
        df = df.reset_index()
        dfs.append(df)
    
    combined = pd.concat(dfs, ignore_index=True)
    print(f"Total events loaded: {len(combined):,}")
    return combined

#####################################
# 2. Feature engineering
#####################################
def add_all_features(df):
    """Create comprehensive order book features."""
    print("\n" + "="*60)
    print("STEP 2: CREATING FEATURES")
    print("="*60)
    features = pd.DataFrame()
    features['date'] = df['date']
    
    # Basic Level-1 features
    features["mid_price"] = (df["BID_PRICE_1"] + df["ASK_PRICE_1"]) / 2
    features["microprice"] = (df["BID_PRICE_1"] * df["ASK_SIZE_1"] + 
                              df["ASK_PRICE_1"] * df["BID_SIZE_1"]) / (df["BID_SIZE_1"] + df["ASK_SIZE_1"] + 1e-10)
    features["spread"] = df["ASK_PRICE_1"] - df["BID_PRICE_1"]
    features["vol_imbalance"] = (df["BID_SIZE_1"] - df["ASK_SIZE_1"]) / (df["BID_SIZE_1"] + df["ASK_SIZE_1"] + 1e-6)
    features["bid_ask_spread_ratio"] = features["spread"] / features["mid_price"]
    
    # All level prices and sizes
    features["BID_PRICE_1"] = df["BID_PRICE_1"]
    features["BID_SIZE_1"] = df["BID_SIZE_1"]
    features["BID_PRICE_2"] = df["BID_PRICE_2"]
    features["BID_SIZE_2"] = df["BID_SIZE_2"]
    features["BID_PRICE_3"] = df["BID_PRICE_3"]
    features["BID_SIZE_3"] = df["BID_SIZE_3"]
    features["ASK_PRICE_1"] = df["ASK_PRICE_1"]
    features["ASK_SIZE_1"] = df["ASK_SIZE_1"]
    features["ASK_PRICE_2"] = df["ASK_PRICE_2"]
    features["ASK_SIZE_2"] = df["ASK_SIZE_2"]
    features["ASK_PRICE_3"] = df["ASK_PRICE_3"]
    features["ASK_SIZE_3"] = df["ASK_SIZE_3"]
    
    # Mean price/quantity across all 3 levels
    features["bid_price_mean"] = (df["BID_PRICE_1"] + df["BID_PRICE_2"] + df["BID_PRICE_3"]) / 3
    features["ask_price_mean"] = (df["ASK_PRICE_1"] + df["ASK_PRICE_2"] + df["ASK_PRICE_3"]) / 3
    features["bid_qty_mean"] = (df["BID_SIZE_1"] + df["BID_SIZE_2"] + df["BID_SIZE_3"]) / 3
    features["ask_qty_mean"] = (df["ASK_SIZE_1"] + df["ASK_SIZE_2"] + df["ASK_SIZE_3"]) / 3
    
    # Cumulative differences across 3 levels
    features["price_cum_diff"] = (df["ASK_PRICE_1"] - df["BID_PRICE_1"] + 
                                  df["ASK_PRICE_2"] - df["BID_PRICE_2"] + 
                                  df["ASK_PRICE_3"] - df["BID_PRICE_3"])
    features["qty_cum_diff"] = (df["ASK_SIZE_1"] - df["BID_SIZE_1"] + 
                                df["ASK_SIZE_2"] - df["BID_SIZE_2"] + 
                                df["ASK_SIZE_3"] - df["BID_SIZE_3"])
    
    # Time intervals between events
    features["time_delta"] = 1  # Placeholder
    
    # Price momentum
    features["mid_diff"] = features["mid_price"].diff()
    features["mid_return"] = features["mid_diff"] / features["mid_price"].shift(1)
    
    # Order Flow Imbalance (OFI)
    features["total_bid_qty"] = df["BID_SIZE_1"] + df["BID_SIZE_2"] + df["BID_SIZE_3"]
    features["total_ask_qty"] = df["ASK_SIZE_1"] + df["ASK_SIZE_2"] + df["ASK_SIZE_3"]
    features["bid_qty_change"] = features["total_bid_qty"].diff()
    features["ask_qty_change"] = features["total_ask_qty"].diff()
    features["OFI"] = features["bid_qty_change"] - features["ask_qty_change"]
    
    # Moving averages
    features["mv_1s"] = features["mid_price"].rolling(1000, min_periods=1).mean()
    features["mv_5s"] = features["mid_price"].rolling(5000, min_periods=1).mean()
    
    # Volatility
    features["vol_10"] = features["mid_return"].rolling(10, min_periods=1).std()
    features["vol_100"] = features["mid_return"].rolling(100, min_periods=1).std()
    features["vol_1s"] = features["mid_return"].rolling(1000, min_periods=1).std()
    features["vol_5s"] = features["mid_return"].rolling(5000, min_periods=1).std()
    
    # RSI
    delta = features["microprice"].diff()
    gain = (delta.where(delta > 0, 0)).rolling(14, min_periods=1).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(14, min_periods=1).mean()
    rs = gain / (loss + 1e-10)
    features["rsi_14"] = 100 - (100 / (1 + rs))
    
    # EMA
    features["ema_fast"] = features["mid_price"].ewm(span=12, adjust=False).mean()
    features["ema_slow"] = features["mid_price"].ewm(span=26, adjust=False).mean()
    features["ema_diff"] = features["ema_fast"] - features["ema_slow"]
    
    # Clean up intermediate columns
    features.drop(['total_bid_qty', 'total_ask_qty', 'bid_qty_change', 'ask_qty_change'], 
                  axis=1, inplace=True, errors='ignore')
    
    # Fill NaN and Inf values
    features = features.ffill().fillna(0)
    features.replace([np.inf, -np.inf], np.nan, inplace=True)
    features = features.ffill().fillna(0)
    
    print(f"Created {len(features.columns)-1} features (excluding 'date')")
    return features

#####################################
# 3. Add labels
#####################################
def add_labels(features, horizon=23):
    """Create price movement labels."""
    print(f"\nCreating labels (horizon = {horizon} events ahead)...")
    features['future_price'] = features['microprice'].shift(-horizon)
    price_change = features['future_price'] - features['microprice']
    
    features['target'] = 1  # neutral (no change)
    features.loc[price_change > 0, 'target'] = 2  # up
    features.loc[price_change < 0, 'target'] = 0  # down
    
    features.drop('future_price', axis=1, inplace=True)
    return features

#####################################
# 4. Feature selection
#####################################
def select_features(df, number_events_ahead=10, max_corr=0.85):
    """Select features using MI scores and Spearman correlation filtering."""
    print("\n" + "="*60)
    print("STEP 3: FEATURE SELECTION")
    print("="*60)
    print(f"Events ahead for MI calculation: {number_events_ahead}")
    print(f"Max correlation threshold: {max_corr}")
    
    # Create temporary label for feature selection
    temp_df = df.copy()
    temp_df["label"] = 0
    temp_df.loc[temp_df["mid_price"].shift(-number_events_ahead) > temp_df["mid_price"], "label"] = 1
    temp_df.loc[temp_df["mid_price"].shift(-number_events_ahead) < temp_df["mid_price"], "label"] = -1

    feature_names = [col for col in temp_df.columns if col not in ['target', 'date', 'label']]
    temp_df = temp_df.dropna()
    
    # Compute MI scores
    print("\nComputing Mutual Information scores...")
    X = temp_df[feature_names].values
    y = temp_df['label'].values
    mi_scores = mutual_info_classif(X, y, random_state=42, n_neighbors=3)
    mi_dict = dict(zip(feature_names, mi_scores))
    
    selected_features = feature_names.copy()
    print(f"Initial features: {len(selected_features)}")
    print("\nIterative correlation-based removal:")
    
    # Iterative removal
    iteration = 0
    while True:
        iteration += 1
        corr_matrix, _ = spearmanr(temp_df[selected_features].values)
        max_correlation = -1
        remove_feat = None
        
        for i in range(len(selected_features)):
            for j in range(i+1, len(selected_features)):
                corr_val = abs(corr_matrix[i, j])
                
                if corr_val > max_corr and corr_val > max_correlation:
                    max_correlation = corr_val
                    feat1 = selected_features[i]
                    feat2 = selected_features[j]
                    
                    if mi_dict[feat1] < mi_dict[feat2]:
                        remove_feat = feat1
                        keep_feat = feat2
                    else:
                        remove_feat = feat2
                        keep_feat = feat1
        
        if remove_feat is None:
            break
        
        print(f"  Iter {iteration}: Removing '{remove_feat}' (corr={max_correlation:.3f} with '{keep_feat}', MI={mi_dict[remove_feat]:.4f})")
        selected_features.remove(remove_feat)
    
    print(f"\nFinal features: {len(selected_features)}")
    print(f"Removed: {len(feature_names) - len(selected_features)} features")

    print("\n" + "="*60)
    print("SELECTED FEATURES WITH MI SCORES (sorted by MI)")
    print("="*60)

    selected_mi = [(feat, mi_dict[feat]) for feat in selected_features]
    selected_mi.sort(key=lambda x: x[1], reverse=True)
    
    for feat, mi_score in selected_mi:
        print(f"{feat:30s} {mi_score:.6f}")
    
    return selected_features

#####################################
# 5. Normalize and split data
#####################################
def normalize_and_split(features, selected_features, train_dates, val_dates, test_dates):
    """Split by date and normalize features."""
    print("\n" + "="*60)
    print("STEP 4: DATA SPLITTING AND NORMALIZATION")
    print("="*60)
    
    train_df = features[features['date'].isin(train_dates)].copy()
    val_df = features[features['date'].isin(val_dates)].copy()
    test_df = features[features['date'].isin(test_dates)].copy()
    
    print(f"Train: {len(train_df):,} events (days {', '.join(train_dates)})")
    print(f"Val:   {len(val_df):,} events (days {', '.join(val_dates)})")
    print(f"Test:  {len(test_df):,} events (days {', '.join(test_dates)})")
    
    train_df = train_df[selected_features + ['target']]
    val_df = val_df[selected_features + ['target']]
    test_df = test_df[selected_features + ['target']]
    
    print("\nNormalizing features using StandardScaler...")
    scaler = StandardScaler()
    scaler.fit(train_df[selected_features])
    
    train_df[selected_features] = scaler.transform(train_df[selected_features])
    val_df[selected_features] = scaler.transform(val_df[selected_features])
    test_df[selected_features] = scaler.transform(test_df[selected_features])
    
    print("\nLabel Distribution:")
    for name, df in [("Train", train_df), ("Val", val_df), ("Test", test_df)]:
        counts = df['target'].value_counts().sort_index()
        total = len(df)
        print(f"\n{name} set ({total:,} samples):")
        print(f"  Down (0):    {counts.get(0, 0):6,} ({counts.get(0, 0)/total*100:5.2f}%)")
        print(f"  Neutral (1): {counts.get(1, 0):6,} ({counts.get(1, 0)/total*100:5.2f}%)")
        print(f"  Up (2):      {counts.get(2, 0):6,} ({counts.get(2, 0)/total*100:5.2f}%)")
    
    X_train, y_train = train_df[selected_features], train_df['target']
    X_val, y_val = val_df[selected_features], val_df['target']
    X_test, y_test = test_df[selected_features], test_df['target']
    
    return X_train, y_train, X_val, y_val, X_test, y_test, scaler

#####################################
# 6. Two-stage hyperparameter tuning (OPTIMIZED)
#####################################
def tune_hyperparameters_two_stage(X_train, y_train, X_val, y_val):
    """Two-stage grid search - BALANCED FOR SPEED & PERFORMANCE."""
    print("\n" + "="*60)
    print("STEP 5: TWO-STAGE HYPERPARAMETER TUNING (BALANCED)")
    print("="*60)
    
    # STAGE 1: REDUCED Coarse search - 32 combinations (~10-15 min)
    print("\n" + "-"*60)
    print("STAGE 1: COARSE GRID SEARCH (REDUCED)")
    print("-"*60)
    
    stage1_grid = {
        'max_depth': [4, 7],                    # 2 options
        'learning_rate': [0.05, 0.15],          # 2 options
        'n_estimators': [150, 250],             # 2 options
        'subsample': [0.8],                     # 1 option (fixed at good default)
        'colsample_bytree': [0.8],              # 1 option (fixed at good default)
        'min_child_weight': [1, 5],             # 2 options
        'gamma': [0, 0.1],                      # 2 options
        'random_state': 42
    }
    
    best_stage1_acc = 0
    best_stage1_params = None
    stage1_history = []
    
    total_stage1 = (len(stage1_grid['max_depth']) * len(stage1_grid['learning_rate']) * 
                    len(stage1_grid['n_estimators']) * len(stage1_grid['subsample']) * 
                    len(stage1_grid['colsample_bytree']) * len(stage1_grid['min_child_weight']) * 
                    len(stage1_grid['gamma']))
    
    print(f"Testing {total_stage1} combinations...\n")
    
    current = 0
    for max_depth in stage1_grid['max_depth']:
        for lr in stage1_grid['learning_rate']:
            for n_est in stage1_grid['n_estimators']:
                for subsample in stage1_grid['subsample']:
                    for colsample in stage1_grid['colsample_bytree']:
                        for min_child in stage1_grid['min_child_weight']:
                            for gamma in stage1_grid['gamma']:
                                current += 1
                                
                                params = {
                                    'objective': 'multi:softmax',
                                    'num_class': 3,
                                    'max_depth': max_depth,
                                    'learning_rate': lr,
                                    'n_estimators': n_est,
                                    'subsample': subsample,
                                    'colsample_bytree': colsample,
                                    'min_child_weight': min_child,
                                    'gamma': gamma,
                                    'random_state': stage1_grid['random_state'],
                                    'eval_metric': 'mlogloss',
                                    'early_stopping_rounds': 20
                                }
                                
                                model = xgb.XGBClassifier(**params)
                                model.fit(X_train, y_train, 
                                         eval_set=[(X_val, y_val)],
                                         verbose=False)
                                
                                y_val_pred = model.predict(X_val)
                                val_acc = accuracy_score(y_val, y_val_pred)
                                precision, recall, f1, _ = precision_recall_fscore_support(
                                    y_val, y_val_pred, average='macro', zero_division=0
                                )
                                
                                print(f"[{current:2d}/{total_stage1}] d={max_depth}, lr={lr:.2f}, n={n_est:3d}, "
                                      f"mcw={min_child}, g={gamma:.1f} | Acc={val_acc:.4f}, F1={f1:.4f}")
                                
                                stage1_history.append({
                                    'max_depth': max_depth,
                                    'learning_rate': lr,
                                    'n_estimators': n_est,
                                    'subsample': subsample,
                                    'colsample_bytree': colsample,
                                    'min_child_weight': min_child,
                                    'gamma': gamma,
                                    'val_accuracy': float(val_acc),
                                    'val_f1': float(f1)
                                })
                                
                                if val_acc > best_stage1_acc:
                                    best_stage1_acc = val_acc
                                    best_stage1_params = params.copy()
    
    print(f"\n{'='*60}")
    print(f"STAGE 1 BEST: Acc={best_stage1_acc:.4f}")
    print(f"  max_depth: {best_stage1_params['max_depth']}")
    print(f"  learning_rate: {best_stage1_params['learning_rate']}")
    print(f"  n_estimators: {best_stage1_params['n_estimators']}")
    print(f"  min_child_weight: {best_stage1_params['min_child_weight']}")
    print(f"  gamma: {best_stage1_params['gamma']}")
    
    # STAGE 2: Fine-tune around best parameters - 81 combinations (~15-20 min)
    print("\n" + "-"*60)
    print("STAGE 2: FINE-TUNING")
    print("-"*60)
    
    # Create refined grid around best parameters
    best_depth = best_stage1_params['max_depth']
    best_lr = best_stage1_params['learning_rate']
    best_n = best_stage1_params['n_estimators']
    
    stage2_grid = {
        'max_depth': [max(3, best_depth-1), best_depth, min(10, best_depth+1)],
        'learning_rate': [max(0.01, best_lr*0.8), best_lr, min(0.5, best_lr*1.2)],
        'n_estimators': [max(50, best_n-50), best_n, best_n+50],
        'subsample': [0.75, 0.8, 0.85],         # 3 options now
        'colsample_bytree': [0.75, 0.8, 0.85],  # 3 options now
        'min_child_weight': [best_stage1_params['min_child_weight']],
        'gamma': [best_stage1_params['gamma']],
        'random_state': 42
    }
    
    best_final_acc = 0
    best_final_params = None
    stage2_history = []
    
    total_stage2 = (len(stage2_grid['max_depth']) * len(stage2_grid['learning_rate']) * 
                    len(stage2_grid['n_estimators']) * len(stage2_grid['subsample']) * 
                    len(stage2_grid['colsample_bytree']))
    
    print(f"Testing {total_stage2} refined combinations...\n")
    
    current = 0
    for max_depth in stage2_grid['max_depth']:
        for lr in stage2_grid['learning_rate']:
            for n_est in stage2_grid['n_estimators']:
                for subsample in stage2_grid['subsample']:
                    for colsample in stage2_grid['colsample_bytree']:
                        current += 1
                        
                        params = {
                            'objective': 'multi:softmax',
                            'num_class': 3,
                            'max_depth': max_depth,
                            'learning_rate': lr,
                            'n_estimators': n_est,
                            'subsample': subsample,
                            'colsample_bytree': colsample,
                            'min_child_weight': stage2_grid['min_child_weight'][0],
                            'gamma': stage2_grid['gamma'][0],
                            'random_state': 42,
                            'eval_metric': 'mlogloss',
                            'early_stopping_rounds': 20
                        }
                        
                        model = xgb.XGBClassifier(**params)
                        model.fit(X_train, y_train,
                                 eval_set=[(X_val, y_val)],
                                 verbose=False)
                        
                        y_val_pred = model.predict(X_val)
                        val_acc = accuracy_score(y_val, y_val_pred)
                        precision, recall, f1, _ = precision_recall_fscore_support(
                            y_val, y_val_pred, average='macro', zero_division=0
                        )
                        
                        if current % 10 == 0 or val_acc > best_final_acc:
                            print(f"[{current:2d}/{total_stage2}] d={max_depth}, lr={lr:.3f}, n={n_est:3d}, "
                                  f"ss={subsample:.2f}, cs={colsample:.2f} | Acc={val_acc:.4f}, F1={f1:.4f}")
                        
                        stage2_history.append({
                            'max_depth': max_depth,
                            'learning_rate': lr,
                            'n_estimators': n_est,
                            'subsample': subsample,
                            'colsample_bytree': colsample,
                            'val_accuracy': float(val_acc),
                            'val_f1': float(f1)
                        })
                        
                        if val_acc > best_final_acc:
                            best_final_acc = val_acc
                            best_final_params = params
    
    print(f"\n{'='*60}")
    print(f"FINAL BEST PARAMETERS (Val Accuracy: {best_final_acc:.4f})")
    print(f"{'='*60}")
    print(f"  max_depth: {best_final_params['max_depth']}")
    print(f"  learning_rate: {best_final_params['learning_rate']:.4f}")
    print(f"  n_estimators: {best_final_params['n_estimators']}")
    print(f"  subsample: {best_final_params['subsample']:.2f}")
    print(f"  colsample_bytree: {best_final_params['colsample_bytree']:.2f}")
    print(f"  min_child_weight: {best_final_params['min_child_weight']}")
    print(f"  gamma: {best_final_params['gamma']}")
    
    full_history = {
        'stage1': stage1_history,
        'stage2': stage2_history,
        'stage1_best': best_stage1_params,
        'stage2_best': best_final_params
    }
    
    return best_final_params, full_history

#####################################
# 7. Train final model
#####################################
def train_final_model(X_train, y_train, X_val, y_val, X_test, y_test, params):
    """Train final XGBoost model with best parameters."""
    print("\n" + "="*60)
    print("STEP 6: TRAINING FINAL MODEL")
    print("="*60)
    
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train,
             eval_set=[(X_val, y_val)],
             verbose=False)
    
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    y_test_pred = model.predict(X_test)
    
    y_train_proba = model.predict_proba(X_train)
    y_val_proba = model.predict_proba(X_val)
    y_test_proba = model.predict_proba(X_test)
    
    results = {}
    
    for name, y_true, y_pred in [
        ('train', y_train, y_train_pred),
        ('val', y_val, y_val_pred),
        ('test', y_test, y_test_pred)
    ]:
        acc = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(
            y_true, y_pred, average='macro', zero_division=0
        )
        
        results[name] = {
            'accuracy': float(acc),
            'precision': float(precision),
            'recall': float(recall),
            'f1': float(f1)
        }
    
    print("\nMODEL PERFORMANCE:")
    print(f"{'='*60}")
    print(f"{'Set':<10} {'Accuracy':<12} {'Precision':<12} {'Recall':<12} {'F1':<12}")
    print(f"{'-'*60}")
    for name in ['train', 'val', 'test']:
        r = results[name]
        print(f"{name.capitalize():<10} {r['accuracy']:.4f}       {r['precision']:.4f}       "
              f"{r['recall']:.4f}       {r['f1']:.4f}")
    
    print(f"\n{'='*60}")
    print("DETAILED TEST SET CLASSIFICATION REPORT")
    print(f"{'='*60}")
    print(classification_report(y_test, y_test_pred, 
                                target_names=['Down (0)', 'Neutral (1)', 'Up (2)']))
    
    print(f"\nConfusion Matrix (Test Set):")
    cm = confusion_matrix(y_test, y_test_pred)
    print(cm)
    
    return model, results, y_train_proba, y_val_proba, y_test_proba

#####################################
# 8. Save outputs
#####################################
def save_outputs(model, results, tuning_history, selected_features, scaler,
                 X_train, X_val, X_test,
                 y_train, y_val, y_test,
                 y_train_proba, y_val_proba, y_test_proba):
    """Save all outputs."""
    print("\n" + "="*60)
    print("STEP 7: SAVING OUTPUTS")
    print("="*60)
    print(f"Saving to: {OUTPUT_DIR}")
    
    model.save_model(str(OUTPUT_DIR / 'xgb_best_model.json'))
    print("✓ xgb_best_model.json")
    
    history = {
        'hyperparameter_tuning': tuning_history,
        'final_results': results,
        'num_features': len(selected_features),
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    with open(OUTPUT_DIR / 'xgb_training_history.json', 'w') as f:
        json.dump(history, f, indent=2)
    print("✓ xgb_training_history.json")
    
    joblib.dump(selected_features, OUTPUT_DIR / 'selected_features.pkl')
    print("✓ selected_features.pkl")
    
    joblib.dump(scaler, OUTPUT_DIR / 'scaler.pkl')
    print("✓ scaler.pkl")
    
    # Save predictions
    train_predictions = pd.DataFrame({
        'actual': y_train.values,
        'predicted': y_train_proba.argmax(axis=1),
        'prob_down': y_train_proba[:, 0],
        'prob_neutral': y_train_proba[:, 1],
        'prob_up': y_train_proba[:, 2]
    })
    train_predictions.to_csv(OUTPUT_DIR / 'xgb_train_predictions.csv', index=False)
    print("✓ xgb_train_predictions.csv")
    
    val_predictions = pd.DataFrame({
        'actual': y_val.values,
        'predicted': y_val_proba.argmax(axis=1),
        'prob_down': y_val_proba[:, 0],
        'prob_neutral': y_val_proba[:, 1],
        'prob_up': y_val_proba[:, 2]
    })
    val_predictions.to_csv(OUTPUT_DIR / 'xgb_val_predictions.csv', index=False)
    print("✓ xgb_val_predictions.csv")
    
    test_predictions = pd.DataFrame({
        'actual': y_test.values,
        'predicted': y_test_proba.argmax(axis=1),
        'prob_down': y_test_proba[:, 0],
        'prob_neutral': y_test_proba[:, 1],
        'prob_up': y_test_proba[:, 2]
    })
    test_predictions.to_csv(OUTPUT_DIR / 'xgb_test_predictions.csv', index=False)
    print("✓ xgb_test_predictions.csv (Day 5 - ready for agent!)")

#####################################
# 9. Main execution
#####################################
if __name__ == "__main__":
    print("="*60)
    print("XGBOOST BALANCED PIPELINE")
    print("Fast Two-Stage Tuning (~30-45 min total)")
    print("="*60)
    
    # Step 1: Load data
    all_dates = ['20251020', '20251021', '20251022', '20251023', '20251024']
    df = fetch_orderbook_data(dates=all_dates)
    
    # Step 2: Create features
    features = add_all_features(df)
    features = add_labels(features, horizon=23)
    
    # Step 3: Feature selection (using training data only)
    train_dates = ['20251020', '20251021', '20251022']
    val_dates = ['20251023']
    test_dates = ['20251024']
    
    train_features = features[features['date'].isin(train_dates)].copy()
    selected_features = select_features(train_features, number_events_ahead=10, max_corr=0.85)
    
    # Step 4: Normalize and split
    X_train, y_train, X_val, y_val, X_test, y_test, scaler = normalize_and_split(
        features, selected_features, train_dates, val_dates, test_dates
    )
    
    # Step 5: Two-stage hyperparameter tuning (BALANCED VERSION)
    best_params, tuning_history = tune_hyperparameters_two_stage(X_train, y_train, X_val, y_val)
    
    # Step 6: Train final model
    model, results, y_train_proba, y_val_proba, y_test_proba = train_final_model(
        X_train, y_train, X_val, y_val, X_test, y_test, best_params
    )
    
    # Step 7: Save everything
    save_outputs(model, results, tuning_history, selected_features, scaler,
                 X_train, X_val, X_test,
                 y_train, y_val, y_test,
                 y_train_proba, y_val_proba, y_test_proba)
    
    print("\n" + "="*60)
    print("PIPELINE COMPLETE!")
    print("="*60)
    print(f"\nModel Performance Summary:")
    print(f"  Test Accuracy: {results['test']['accuracy']:.4f}")
    print(f"  Test F1 Score: {results['test']['f1']:.4f}")
    print(f"  Features Used: {len(selected_features)}")
    print(f"\nAll files saved to: {OUTPUT_DIR}")
    print(f"\nKey output: xgb_test_predictions.csv (ready for agent!)")
    
    # Feature importance analysis
    print("\n" + "="*60)
    print("TOP 10 MOST IMPORTANT FEATURES")
    print("="*60)
    
    feature_importance = model.feature_importances_
    importance_df = pd.DataFrame({
        'feature': selected_features,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    for idx, row in importance_df.head(10).iterrows():
        print(f"{row['feature']:30s} {row['importance']:.6f}")
    
    # Save feature importance
    importance_df.to_csv(OUTPUT_DIR / 'feature_importance.csv', index=False)
    print("\n✓ feature_importance.csv saved")
    
    print("\n" + "="*60)
    print("TUNING SUMMARY")
    print("="*60)
    print(f"Stage 1: Tested 32 combinations (coarse search)")
    print(f"Stage 2: Tested 81 combinations (fine-tuning)")
    print(f"Total combinations evaluated: 113")
    print(f"\nEstimated runtime: 30-45 minutes")
    print("="*60)

XGBOOST BALANCED PIPELINE
Fast Two-Stage Tuning (~30-45 min total)
STEP 1: LOADING RAW ORDER BOOK DATA
  Loading 20251020...
  Loading 20251021...
  Loading 20251022...
  Loading 20251023...
  Loading 20251024...
Total events loaded: 5,583,093

STEP 2: CREATING FEATURES
Created 37 features (excluding 'date')

Creating labels (horizon = 23 events ahead)...

STEP 3: FEATURE SELECTION
Events ahead for MI calculation: 10
Max correlation threshold: 0.85

Computing Mutual Information scores...
Initial features: 37

Iterative correlation-based removal:
  Iter 1: Removing 'ema_slow' (corr=1.000 with 'ema_fast', MI=0.0406)
  Iter 2: Removing 'mid_diff' (corr=0.999 with 'mid_return', MI=0.0194)
  Iter 3: Removing 'mid_price' (corr=0.999 with 'microprice', MI=0.0105)
  Iter 4: Removing 'mv_1s' (corr=0.996 with 'ema_fast', MI=0.0144)
  Iter 5: Removing 'ema_fast' (corr=0.996 with 'microprice', MI=0.0609)
  Iter 6: Removing 'ASK_PRICE_1' (corr=0.996 with 'microprice', MI=0.0126)
  Iter 7: Removing 